## 1) Configurar entorno y rutas
Importar librerías y definir rutas de entrada/salida en la carpeta `proyecto_lu`.

In [1]:
import pandas as pd
from pathlib import Path
import os

# Rutas (relativas al repo raíz)
repo_root = Path('.')
input_path = repo_root / 'proyecto_lu' / 'data_with_climate.csv'
output_path = repo_root / 'proyecto_lu' / 'data_with_climate_fixed.csv'
backup_path = repo_root / 'proyecto_lu' / 'data_with_climate_backup.csv'

# Asegurar que la carpeta exista
(repo_root / 'proyecto_lu').mkdir(parents=True, exist_ok=True)

print('Input path:', input_path)
print('Output path:', output_path)
print('Backup path:', backup_path)

Input path: proyecto_lu\data_with_climate.csv
Output path: proyecto_lu\data_with_climate_fixed.csv
Backup path: proyecto_lu\data_with_climate_backup.csv


## 2) Cargar CSV con pandas
Leer el CSV; parsear `fecha` como datetime. Para `Date (YMD)` haremos el parse posterior porque su formato es `d/m/Y`.

In [7]:
# Leemos sin forzar tipos para evitar errores de parseo iniciales
df = pd.read_csv("data_with_climate.csv", dtype=str)

# Parsear columna 'fecha' (si está presente)
if 'fecha' in df.columns:
    df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')
else:
    df['fecha'] = pd.NaT

print('Shape:', df.shape)
df.head(5)

Shape: (48111, 18)


,Date (YMD),Provincia,Code Provincia,Departamento,Code Departamento,lat,lon,hayRioCercano,distanciaRio,codeRio,tipoDeSuelo,estacion,fecha,surface_runoff_sum,temperature_2m,total_evaporation_sum,total_precipitation_sum,volumetric_soil_water_layer_1
0,29/6/2014,Misiones,54,Capital,54028,-27.47659493,-55.92169961,True,8.07,14-14,Alfisoles,invierno,2014-06-23,0.0050834715366363525,291.45590527852374,-0.0011485274881124496,0.04183991253376007,0.46769078572591144
1,29/6/2014,Misiones,54,Capital,54028,-27.47659493,-55.92169961,True,8.07,14-14,Alfisoles,invierno,2014-06-24,NaN,NaN,NaN,NaN,NaN
2,29/6/2014,Misiones,54,Capital,54028,-27.47659493,-55.92169961,True,8.07,14-14,Alfisoles,invierno,2014-06-25,NaN,NaN,NaN,NaN,NaN
3,29/6/2014,Misiones,54,Capital,54028,-27.47659493,-55.92169961,True,8.07,14-14,Alfisoles,invierno,2014-06-26,NaN,NaN,NaN,NaN,NaN
4,29/6/2014,Misiones,54,Capital,54028,-27.47659493,-55.92169961,True,8.07,14-14,Alfisoles,invierno,2014-06-27,NaN,NaN,NaN,NaN,NaN


## 3) Inspeccionar columnas y filas relevantes
Mostramos información básica y las columnas de interés.

In [11]:
# Información general
print(df.columns.tolist())

print(df.info())

# Mostrar valores únicos (muestra) de las columnas de fecha
print('Unique Date (YMD) sample:', df['Date (YMD)'].dropna().unique()[:5])
print('Unique fecha sample:', df['fecha'].dropna().astype(str).unique()[:5])

['Date (YMD)', 'Provincia', 'Code Provincia', 'Departamento', 'Code Departamento', 'lat', 'lon', 'hayRioCercano', 'distanciaRio', 'codeRio', 'tipoDeSuelo', 'estacion', 'fecha', 'surface_runoff_sum', 'temperature_2m', 'total_evaporation_sum', 'total_precipitation_sum', 'volumetric_soil_water_layer_1']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48111 entries, 0 to 48110
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Date (YMD)                     48111 non-null  object        
 1   Provincia                      48111 non-null  object        
 2   Code Provincia                 48111 non-null  object        
 3   Departamento                   48111 non-null  object        
 4   Code Departamento              48111 non-null  object        
 5   lat                            48111 non-null  object        
 6   lon                            48111 non-null  o

## 4) Identificar filas 'de datos' y primera fecha
Definimos las columnas de datos climáticos y detectamos filas con datos (no-null). También mostramos dos criterios: `any` (cualquier columna climática tiene valor) y `all` (todas las columnas no-fecha tienen valor).

In [12]:
# Columnas de interés (climáticas) - ajustar si en tu CSV hay otras
climate_cols = [
    'surface_runoff_sum', 'temperature_2m', 'total_evaporation_sum',
    'total_precipitation_sum', 'volumetric_soil_water_layer_1'
]
# Filtrar solo las columnas que existan en el DataFrame
climate_cols = [c for c in climate_cols if c in df.columns]
print('Climate columns used:', climate_cols)

# Considerar otras columnas de datos (según outline): todas excepto 'Date (YMD)' y 'fecha'
columns_de_datos = [c for c in df.columns if c not in ['Date (YMD)', 'fecha']]

# Detectar filas con datos según dos criterios
filas_de_datos_any = df[climate_cols].notna().any(axis=1) if climate_cols else pd.Series(False, index=df.index)
filas_de_datos_all = df[columns_de_datos].notna().all(axis=1)

print('Filas con al menos una columna climática no-nula (any):', filas_de_datos_any.sum())
print('Filas con todas las columnas no-fecha no-nulas (all):', filas_de_datos_all.sum())

# Parsear Date (YMD) con dayfirst=True
df['Date_parsed'] = pd.to_datetime(df['Date (YMD)'], dayfirst=True, errors='coerce')

# Mostrar ejemplos donde hay datos climáticos y Date_parsed existe
sample_idx = df[filas_de_datos_any & df['Date_parsed'].notna()].index[:5]
df.loc[sample_idx, ['Date (YMD)', 'Date_parsed', 'fecha']]

Climate columns used: ['surface_runoff_sum', 'temperature_2m', 'total_evaporation_sum', 'total_precipitation_sum', 'volumetric_soil_water_layer_1']
Filas con al menos una columna climática no-nula (any): 6689
Filas con todas las columnas no-fecha no-nulas (all): 6689


,Date (YMD),Date_parsed,fecha
0,29/6/2014,2014-06-29,2014-06-23
14,28/10/2014,2014-10-28,2014-10-27
21,12/2/2014,2014-02-12,2014-02-10
28,2/3/2015,2015-03-02,2015-03-02
35,2/3/2015,2015-03-02,2015-03-02


## 5) Reemplazar `Date (YMD)` por `fecha` para filas de datos
Haremos la asignación sólo en las filas donde detectamos datos (usando `any`) y cuando las fechas difieran. También guardamos una copia de seguridad.

In [13]:
# Guardar backup rápido
df.to_csv(backup_path, index=False)
print('Backup guardado en', backup_path)

# Preparar comparación segura (formato ISO)
df['fecha_iso'] = df['fecha'].dt.strftime('%Y-%m-%d')
df['date_parsed_iso'] = df['Date_parsed'].dt.strftime('%Y-%m-%d')

# Mascara de reemplazo: filas con datos climáticos (any) y donde date_parsed_iso != fecha_iso (y ambas no son NaN)
replaced_mask = filas_de_datos_any & df['fecha_iso'].notna() & (df['date_parsed_iso'] != df['fecha_iso'])
replacements = replaced_mask.sum()
print('Reemplazos a realizar (filas):', replacements)

# Aplicar reemplazo: asignamos la representación ISO de 'fecha' a la columna 'Date (YMD)'
# (Si prefieres conservar el formato original d/m/Y, habría que formatearlo. Aquí usamos ISO para consistencia.)
df.loc[replaced_mask, 'Date (YMD)'] = df.loc[replaced_mask, 'fecha_iso']

# Crear columna request_date: para filas con datos usamos 'fecha', si no usamos Date_parsed (ISO)
df['request_date'] = df['fecha_iso']
mask_no_fecha = df['request_date'].isna()
df.loc[mask_no_fecha, 'request_date'] = df.loc[mask_no_fecha, 'date_parsed_iso']

print('Primeras filas modificadas (ejemplo):')
df.loc[replaced_mask, ['Date (YMD)', 'date_parsed_iso', 'fecha_iso', 'request_date']].head(10)

Backup guardado en proyecto_lu\data_with_climate_backup.csv
Reemplazos a realizar (filas): 5669
Primeras filas modificadas (ejemplo):


,Date (YMD),date_parsed_iso,fecha_iso,request_date
0,2014-06-23,2014-06-29,2014-06-23,2014-06-23
14,2014-10-27,2014-10-28,2014-10-27,2014-10-27
21,2014-02-10,2014-02-12,2014-02-10,2014-02-10
49,2015-02-23,2015-02-25,2015-02-23,2015-02-23
56,2015-02-23,2015-02-25,2015-02-23,2015-02-23
63,2015-02-23,2015-02-25,2015-02-23,2015-02-23
70,2015-02-23,2015-02-25,2015-02-23,2015-02-23
77,2012-09-03,2012-09-06,2012-09-03,2012-09-03
98,2015-01-12,2015-01-13,2015-01-12,2015-01-12
105,2015-01-12,2015-01-13,2015-01-12,2015-01-12


## 6) Validar cambios y pruebas simples
Comprobaciones básicas: conteo de reemplazos y asserts simples.

In [15]:
# Contar reemplazos hechos (nuevamente, para confirmar)
replaced_after = (df['date_parsed_iso'] != df['fecha_iso']) & filas_de_datos_any
# nota: date_parsed_iso no cambia automáticamente; recomputamos si queremos.
# Verificamos que en las filas de datos, request_date coincide con fecha_iso
if filas_de_datos_any.any():
    assert (df.loc[filas_de_datos_any, 'request_date'] == df.loc[filas_de_datos_any, 'fecha_iso']).all(),
    'En filas de datos, request_date debe ser igual a fecha_iso'

print('Asserts pasaron. Filas marcadas como datos (any):', filas_de_datos_any.sum())
print('Reemplazos aplicados (replaced_mask):', replacements)

# Mostrar algunas filas finales como verificación
display(df.head(10))

Asserts pasaron. Filas marcadas como datos (any): 6689
Reemplazos aplicados (replaced_mask): 5669


,Date (YMD),Provincia,Code Provincia,Departamento,Code Departamento,lat,lon,hayRioCercano,distanciaRio,codeRio,...,fecha,surface_runoff_sum,temperature_2m,total_evaporation_sum,total_precipitation_sum,volumetric_soil_water_layer_1,Date_parsed,fecha_iso,date_parsed_iso,request_date
0,2014-06-23,Misiones,54,Capital,54028,-27.47659493,-55.92169961,True,8.07,14-14,...,2014-06-23,0.0050834715366363525,291.45590527852374,-0.0011485274881124496,0.04183991253376007,0.46769078572591144,2014-06-29,2014-06-23,2014-06-29,2014-06-23
1,29/6/2014,Misiones,54,Capital,54028,-27.47659493,-55.92169961,True,8.07,14-14,...,2014-06-24,NaN,NaN,NaN,NaN,NaN,2014-06-29,2014-06-24,2014-06-29,2014-06-24
2,29/6/2014,Misiones,54,Capital,54028,-27.47659493,-55.92169961,True,8.07,14-14,...,2014-06-25,NaN,NaN,NaN,NaN,NaN,2014-06-29,2014-06-25,2014-06-29,2014-06-25
3,29/6/2014,Misiones,54,Capital,54028,-27.47659493,-55.92169961,True,8.07,14-14,...,2014-06-26,NaN,NaN,NaN,NaN,NaN,2014-06-29,2014-06-26,2014-06-29,2014-06-26
4,29/6/2014,Misiones,54,Capital,54028,-27.47659493,-55.92169961,True,8.07,14-14,...,2014-06-27,NaN,NaN,NaN,NaN,NaN,2014-06-29,2014-06-27,2014-06-29,2014-06-27
5,29/6/2014,Misiones,54,Capital,54028,-27.47659493,-55.92169961,True,8.07,14-14,...,2014-06-28,NaN,NaN,NaN,NaN,NaN,2014-06-29,2014-06-28,2014-06-29,2014-06-28
6,29/6/2014,Misiones,54,Capital,54028,-27.47659493,-55.92169961,True,8.07,14-14,...,2014-06-29,NaN,NaN,NaN,NaN,NaN,2014-06-29,2014-06-29,2014-06-29,2014-06-29
7,6/2/2014,Buenos Aires,6,Berisso,06098,-34.84587606,-57.87973336,True,0.7,3314-1708,...,2014-02-03,NaN,NaN,NaN,NaN,NaN,2014-02-06,2014-02-03,2014-02-06,2014-02-03
8,6/2/2014,Buenos Aires,6,Berisso,06098,-34.84587606,-57.87973336,True,0.7,3314-1708,...,2014-02-04,NaN,NaN,NaN,NaN,NaN,2014-02-06,2014-02-04,2014-02-06,2014-02-04
9,6/2/2014,Buenos Aires,6,Berisso,06098,-34.84587606,-57.87973336,True,0.7,3314-1708,...,2014-02-05,NaN,NaN,NaN,NaN,NaN,2014-02-06,2014-02-05,2014-02-06,2014-02-05


## 7) Guardar CSV corregido
Guardamos el CSV corregido en `proyecto_lu/data_with_climate_fixed.csv`.

In [ ]:
df.to_csv(output_path, index=False)
print('Archivo corregido guardado en', output_path)

# Mostrar ruta absoluta para conveniencia
print('Absolute path:', (output_path).resolve())

---
### Notas finales
- El notebook realiza el reemplazo cuando detecta datos climáticos en las filas (criterio `any`).
- Si prefieres usar el criterio `all` (todas las columnas no-fecha no nulas) modifica `replaced_mask` a `filas_de_datos_all & ...`.
- La columna `request_date` es la recomendada para usar en llamadas a APIs: contiene la `fecha` (cuando hay datos) o la fecha parseada desde `Date (YMD)` en caso contrario.